In [1]:
from fastai.text.all import *

In [2]:
base_path = Path('src')

img_train = base_path / 'train_images'
img_test = base_path / 'test_images'

In [3]:
df_train = pd.read_csv(base_path / 'train.csv')
df_test = pd.read_csv(base_path / 'test.csv')

In [8]:
df = pd.concat([df_train.drop(['label_group'], axis = 1), df_test]).drop(['posting_id','image','image_phash'], axis = 1)

In [10]:
df.head()

,title
0,Paper Bag Victoria Secret
1,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE"
2,Maling TTS Canned Pork Luncheon Meat 397 gr
3,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
4,Nescafe \xc3\x89clair Latte 220ml


In [19]:
dls_lm = DataBlock(
    blocks = TextBlock.from_df(text_cols='title', is_lm = True, min_freq = 1, max_vocab = None),
    get_items = ColReader('title'),
    splitter = RandomSplitter(0.1),    
).dataloaders(df, bs = 32, seq_len = 80)

In [20]:
b = dls_lm.one_batch()
vocab = dls_lm.vocab

In [21]:
dls_lm.show_batch(max_n = 2)

,text,text_
0,xxbos xxmaj headset xxmaj philips xxup at-036 xxmaj bass+ xxmaj magnetic xxmaj mic xxmaj bass xxmaj in xxmaj ear xxmaj stereo xxmaj earphone xxbos 100 xxup pcs xxmaj ikat xxmaj karet xxmaj rambut xxmaj elastis xxmaj gaya xxmaj korea - xxmaj aksesoris xxmaj rambut xxbos xxmaj casing xxmaj iphone 11 7plus 8plus 11pro xxmaj max iphone 12 xxmaj pro xxmaj max iphone 12 xxmaj mini xxmaj iphone 8 7 xxmaj plus 6 6s xxmaj plus x xxmaj xs xxmaj max,xxmaj headset xxmaj philips xxup at-036 xxmaj bass+ xxmaj magnetic xxmaj mic xxmaj bass xxmaj in xxmaj ear xxmaj stereo xxmaj earphone xxbos 100 xxup pcs xxmaj ikat xxmaj karet xxmaj rambut xxmaj elastis xxmaj gaya xxmaj korea - xxmaj aksesoris xxmaj rambut xxbos xxmaj casing xxmaj iphone 11 7plus 8plus 11pro xxmaj max iphone 12 xxmaj pro xxmaj max iphone 12 xxmaj mini xxmaj iphone 8 7 xxmaj plus 6 6s xxmaj plus x xxmaj xs xxmaj max xxmaj
1,"sen xxup motor xxup super xxup bright xxbos xxup mama xxup lemon xxup jeruk xxup nipis xxup refill 780ml xxbos xxmaj lion xxmaj star xxmaj hugo 103 xxmaj toples xxmaj round xxmaj jar 6 xxmaj liter xxbos xxmaj karpet xxmaj refleksi xxmaj pijat xxmaj telapak xxmaj kaki xxbos xxup rak xxmaj dinding 3 xxup susun xxmaj gantung xxup rak xxup kosmetik xxup shabby xxup vintage , rak hp remote xxup tv xxbos xxmaj oppo xxup a37 ram 2gb-16 gb garansi","xxup motor xxup super xxup bright xxbos xxup mama xxup lemon xxup jeruk xxup nipis xxup refill 780ml xxbos xxmaj lion xxmaj star xxmaj hugo 103 xxmaj toples xxmaj round xxmaj jar 6 xxmaj liter xxbos xxmaj karpet xxmaj refleksi xxmaj pijat xxmaj telapak xxmaj kaki xxbos xxup rak xxmaj dinding 3 xxup susun xxmaj gantung xxup rak xxup kosmetik xxup shabby xxup vintage , rak hp remote xxup tv xxbos xxmaj oppo xxup a37 ram 2gb-16 gb garansi toko"


In [42]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult = 0.7,
    metrics = [accuracy, Perplexity()]
).to_fp16()

In [43]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.143374,3.684131,0.410375,39.810516,00:31


In [44]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.460683,3.549863,0.424141,34.808537,00:31
1,3.247396,3.371286,0.444859,29.115957,00:31
2,2.973815,3.229550,0.462750,25.268288,00:31
3,2.668709,3.150180,0.477016,23.340261,00:31
4,2.412720,3.117157,0.486547,22.582081,00:31
5,2.166969,3.102320,0.495531,22.249510,00:31
6,1.984233,3.115340,0.502125,22.541098,00:31
7,1.822430,3.126861,0.505953,22.802298,00:31
8,1.731538,3.144025,0.507547,23.197056,00:32
9,1.686211,3.151995,0.507609,23.382660,00:31


## Save fine tuned LM & vocab for Shopee Dataset

In [45]:
learn.save_encoder('shopee_LM_finetuned_dropout0.7')

In [31]:
with open('models/shopee_vocab', 'wb') as f:
    pickle.dump(vocab, f)